![image](../images/gnod_2nd_iteration.jpg)

In [101]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from getpass import getpass
from tqdm.notebook import tqdm
import pandas as pd
import joblib

In [ ]:
#hot_100 = pd.read_csv("../data/top-100-songs-chart.csv", index_col = 0)

In [59]:
def first_input ():
    """
    inputs a string of at least 2 characters, returns that string stripped from whitespace
    """
    inp = str(input("Insert the name of a song: ")).strip()
    while len(inp)<2:
        inp = str(input("please write at least 2 characters as a name: ")).strip()
    return inp

In [43]:
def matching_songs(inp,song,artists,songid):
    """
    creates a dframe of songs where inp matches a song name
    will return a list with song, artist and an index for these songs 
    """
    
    #We create a list of songs where the inputted string matches
    matches = song.str.lower().str.find(inp.lower())
    songs = [(song[i],artists[i],songid[i]) for i,x in enumerate(matches) if x >= 0]
    
    #If there is no matches whatsoever we return false, if any match, return songs list 
    if len(songs) == 0:
        return False
    else:
        return songs
    



In [61]:
def select_song(songs):
    """
    will return the index of the original chart in where the song is located, false if no matches. 
    """
    
    if len(songs) == 1:    
        print("\nDid you mean this song?")
    else:
        print("\nDid you mean any of these?")
        
    #displaying the options and ask for selection
    for i,song in enumerate(songs):
        print(str(i),"-> ", song[0]," | ",song[1])    
    
    sel = input("Choose song if yes, N if not: ").strip()
    
    #we use try/except to raise errors if the index is out of range, or if the input can't be converted to an int 
    while sel.lower() != "n":
        try:
            return songs[int(sel)][2]
            break
        except ValueError:
            sel = input("Sorry, I didn't understand! Choose number if yes, N if not.").strip()
        except IndexError:
            sel = input("Please choose one of the songs or write N if not: ").strip()
    
    return False 

In [53]:
def hot_songs(inp):
    """
    requires pandas
    """
    hot_100 = pd.read_csv("../data/top-100-songs-chart.csv", index_col = 0)
    
    matching = matching_songs(inp,hot_100['song'],hot_100['artist'],hot_100['rank'])
    
    if matching == False:
        return False
    
    song_ind = select_song(matching)
    
    if song_ind == False:
        return False 
    else:
        recommendation = hot_100[hot_100['rank'] != song_ind].sample()
        song = recommendation['song'].iloc[0]
        artist = recommendation['artist'].iloc[0]
        print("\nWe recommend you {} from {} ".format(song,artist))
        return True

In [16]:
def spotipy_connection():
    print("Insert Spotipy Client ID:")
    c_id = str(getpass())
    print("Insert Spotipy Client Secret:")
    c_s = str(getpass())
    
    
    return c_id,c_s

In [17]:
c_id,c_s = spotipy_connection()

Insert Spotipy Client ID:
········
Insert Spotipy Client Secret:
········


In [18]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,
                                                          client_secret=c_s))

In [40]:
def spotify_query(inp,offs = 0):
    #queries 10 song names to spotify api
    tracks_query = sp.search(str(inp), limit=10, type='track', offset=offs)['tracks']['items']
    songs = []
    for track in tracks_query:
        sp_song = track['name']
        sp_artists = ", ".join([artist['name'] for artist in track['artists']])
        sp_id = track['id']
        songs.append((sp_song,sp_artists,sp_id))
    return songs
    
    

In [117]:
def spotify_inp(inp):
    #Tis can be improved to roll in various pages of the query, or by asking a new input
    sp_songs = spotify_query(inp)
    
    selection = select_song(sp_songs)

    return selection
   

In [129]:
def spotify_recommender(song_id):
    scaler = joblib.load('../data/model/std_scaler.bin')
    model = joblib.load('../data/model/kmeans_model.sav')
    cluster_id = pd.load_csv()
    
    song_feats = sp.audio_features(song_id)[0]
    features = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'] 
    result = pd.DataFrame({key:song_feats[key] for key in features},index = [0])
    scaler = joblib.load('../data/model/std_scaler.bin')
    
    return scaler.transform(result)

In [131]:
feat = spotify_features("3J5jotqUAjU479WhAiNkpU")

In [107]:
scaler = joblib.load('../data/model/std_scaler.bin')

In [109]:
scaler.transform(a)

array([[-0.45434349,  0.0449069 ,  0.78410866,  0.68339756, -0.44716746,
         0.27120376, -0.5720295 , -0.81486294,  0.47914816,  1.96925457]])

In [121]:
model = joblib.load('../data/model/kmeans_model.sav')

In [132]:
model.predict(feat)

array([17])

In [ ]:
series = pd.DataFrame({key:song_feats[key] for key in features},index = [0])

In [119]:
def final():
    
    # we take the first input 
    inp = first_input()
    # check if a hot song 
    if hot_songs(inp):
        print("Finished after hot songs recommendation")
    
    song_id = spotify_inp(inp)
    
    feats = spotify_features(song_id)
    
    
    
    
    
    

In [120]:
final()

Insert the name of a song: highway

Did you mean any of these?
0 ->  Highway to Hell  |  AC/DC
1 ->  Highwayman  |  The Highwaymen, Willie Nelson, Johnny Cash, Waylon Jennings, Kris Kristofferson
2 ->  Life is a Highway  |  Rascal Flatts
3 ->  Highway Don't Care  |  Tim McGraw, Taylor Swift, Keith Urban
4 ->  Highway Tune  |  Greta Van Fleet
5 ->  Ventura Highway  |  America, George Martin
6 ->  Highway 20 Ride  |  Zac Brown Band
7 ->  Highway Star  |  Deep Purple
8 ->  Highway 40 Blues (feat. Ricky Skaggs & Larry Cordle)  |  Tyler Childers, Ricky Skaggs, Larry Cordle
9 ->  Freedom Was A Highway  |  Jimmie Allen, Brad Paisley
Choose song if yes, N if not: 9
[[-0.0067472   1.21552015 -1.18791259  0.87424238 -0.42457607 -0.96525103
  -0.5720295  -0.34059966  0.6909589  -0.69824669]]


### not being used now

In [ ]:
hot_100[hot_100['rank']!=2]

In [5]:
def check_input_song(songs):
    """
    Takes a dataframe of song matches and will ask to confirm selection based on those possible matches.
    
    """
    for i,song in enumerate(songs[:10]):
        print(str(i),"-> ", song[0]," | ",song[1])    
    
    sel = input("Choose song if yes, N if not: ").strip()
    
    #we use try/except to raise errors if the index is out of range, or if the input can't be converted to an int 
    while sel.lower() != "n":
        try:
            return songs[int(sel)][2]
            break
        except ValueError:
            sel = input("Sorry, I didn't understand! Choose number if yes, N if not.").strip()
        except IndexError:
            sel = input("Please choose one of the songs or write N if not: ").strip()
    
    return False